## Make demographic dataframes for top3, bottom3 and nominated counties

#### By Xinqian Zhai, Cliff Gong, Gen Ho 

In [1]:
# import libararies
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import altair as alt

### Define the nominated, the top3, and the bottom3 (lower-ranked) counties
- The nominated counties are the counties with the highest ROI (ROI >7%) over all counties of three nominated states, CA, NY, and TX. 
- The top3 are the three counties with the highest ROI in each nominated state. 
- The bottom3 are the three counties with the lowest ROI in each nominated state.

### Finding the nominated, the top3, and the bottom3 counties 

Now we show how we create the above variables in code.

In [2]:
df_merge = pd.read_csv('./output/merge.csv')

In [3]:
df_merge.head()

,date,county,selling_price,state,rent,roi,rate
0,2018-01-01,Alameda County,840482.0,CA,2541.0,-1.53,0.0395
1,2018-02-01,Alameda County,853351.0,CA,2545.0,-2.32,0.0422
2,2018-03-01,Alameda County,862162.0,CA,2550.0,-2.91,0.0443
3,2018-04-01,Alameda County,868163.0,CA,2555.0,-2.89,0.0440
4,2018-05-01,Alameda County,877878.0,CA,2560.0,-3.35,0.0455


In [4]:
# take a look at the top 10 counties with the highest ROI
df_merge.groupby(['state','county']).median().round(3).sort_values(by=['roi'],ascending=False)[:10]

selling_price    rent     roi   rate
state county                                                     
TX    El Paso County              136391.0  1101.0  10.030  0.037
      Kaufman County              229317.5  1652.0   9.145  0.037
      Johnson County              225958.5  1545.0   8.910  0.037
      Harris County               201007.5  1417.5   7.980  0.037
NY    Bronx County                311455.5  1814.5   7.965  0.037
TX    Dallas County               221054.0  1446.0   6.920  0.037
CA    San Bernardino County       355096.5  1905.0   6.710  0.037
TX    Tarrant County              223225.5  1462.0   6.365  0.037
      Guadalupe County            238859.0  1463.0   6.365  0.037
      Bexar County                194048.0  1200.0   5.575  0.037

Ok df_merge, one of our master tables that have been cleaned, loaded correctly. Now let's create those variables.

In [5]:
def get_median_county_data_by_state(state):
    group = df_merge[df_merge['state']==state].groupby('county')
    return group.median().round(3).sort_values(by=['roi'],ascending=False)

# Example, show top 5 ROI for CA
get_median_county_data_by_state('CA')[:5]

,selling_price,rent,roi,rate
county,,,,
San Bernardino County,355096.5,1905.0,6.710,0.037
Riverside County,395686.5,1989.0,5.260,0.037
San Joaquin County,373617.5,1796.5,4.835,0.037
Sacramento County,376732.5,1706.5,3.915,0.037
Ventura County,614265.0,2669.0,3.750,0.037


In [6]:
def get_counties_of_interest(n_items):
    nominated_states = ['CA','NY','TX']
    counties = list()
    for state in nominated_states:
        if n_items >= 0:
            tmp_counties = list(get_median_county_data_by_state(state)[:n_items].index.values)
        else:
            tmp_counties = list(get_median_county_data_by_state(state)[n_items:].index.values)
        counties += [s + ',{}'.format(state) for s in tmp_counties]
    
    return counties

In [7]:
# top 3 in each nominated state
top3_counties = get_counties_of_interest(3)
top3_counties

['San Bernardino County,CA',
 'Riverside County,CA',
 'San Joaquin County,CA',
 'Bronx County,NY',
 'Orange County,NY',
 'Monroe County,NY',
 'El Paso County,TX',
 'Kaufman County,TX',
 'Johnson County,TX']

In [8]:
# Bottom 3 in each nominated state
bottom3_counties = get_counties_of_interest(-3)
bottom3_counties

['San Francisco County,CA',
 'Santa Clara County,CA',
 'San Mateo County,CA',
 'Westchester County,NY',
 'New York County,NY',
 'Dutchess County,NY',
 'Hays County,TX',
 'Collin County,TX',
 'Travis County,TX']

In [9]:
def get_nominated_counties(threshold):
    nom_df = df_merge.groupby(['state','county']).median().round(3).sort_values(by=['roi'],ascending=False)
    nom_df = nom_df[nom_df['roi'] >= threshold]
    nom_counties = list()
    for item in nom_df.index:
        s = item[1] + ',' + item[0]
        nom_counties.append(s)
    
    return nom_counties

In [10]:
# nominated counties that have ROI > 7(%)
nominated_counties = get_nominated_counties(7.0)
nominated_counties

['El Paso County,TX',
 'Kaufman County,TX',
 'Johnson County,TX',
 'Harris County,TX',
 'Bronx County,NY']

#### Findings about nominated counties (ROI > 7%):

As we can see, 4 out of 5 nominated counties with the highest ROI are all from Texas,  and the other county is from New York state. However, in California, the most populous state, no county has reached the 7% ROI threshold.
So based on the data, **Texas turns out to be a good place for rental property investors who aim for steady and high cash flow.**

### Make demographic dataframes for the nominated, the top3, and the bottom3 counties

In [11]:
# load demographic data from demographic_data
bottom3_dem_df = pd.read_csv("./demographic_data/bottom3_demographic.csv", index_col=0)
nominated_dem_df = pd.read_csv("./demographic_data/nominated_demographic.csv", index_col=0)
top3_dem_df = pd.read_csv("./demographic_data/top3_demographic.csv", index_col=0)

In [12]:
# get needed demographic facotrs
def get_dem(df):
    dem_df = df.copy()
    # transpose the rows and columns
    dem_df = dem_df.T.reset_index()
    dem_df = dem_df.drop(columns = ['Occupied Housing Units','Total Housing Units','Total Population','Average Household Income', 'Total Households'])
    dem_df.rename(columns = {'index': 'county', 'Population Density (per Sq. Mi.)': 'Population Density'}, inplace=True)
    # change the column name to align with population data
    dem_df['county'] = dem_df['county'].str.split(',').str.join(' County,')

    return dem_df

bottom3_dem = get_dem(bottom3_dem_df)
nominated_dem = get_dem(nominated_dem_df)

# add population growth rate data to demographic data

# load population data from additional_output
pop_growth_rate = pd.read_csv("./demographic_data/pop_growth_rate.csv")
pop_growth_rate = pop_growth_rate.rename({'growth_rate': 'Population Growth'}, axis=1)
# keep only the bottom3 and nominated counties
bottom3_pop_growth_rate = pop_growth_rate[pop_growth_rate['county'].isin(bottom3_counties)].iloc[:, [0,-1]]
nominated_pop_growth_rate = pop_growth_rate[pop_growth_rate['county'].isin(nominated_counties)].iloc[:, [0,-1]]
# merge bottom3, nominated with population growth rate together
bottom3_dem = bottom3_dem.merge(bottom3_pop_growth_rate, on='county')
nominated_dem = nominated_dem.merge(nominated_pop_growth_rate, on='county')


In [13]:
# print(bottom3_dem.shape)
bottom3_dem.head()

,county,Population Density,Median Age,Vacant Housing Units,Median Household Income,Population Growth
0,"San Francisco County,CA",18827.29,38.3,0.06,120229.0,0.003
1,"San Mateo County,CA",1669.67,39.9,0.01,132759.0,-0.008
2,"Santa Clara County,CA",1489.51,37.4,0.03,135085.0,-0.007
3,"Dutchess County,NY",358.98,42.5,0.11,83218.0,0.003
4,"New York County,NY",57255.99,37.7,0.12,90365.0,-0.001


Make a bar chart using altair to further see the diffences of each demographic factor at county level

In [14]:
def plot_dem_features(df, feature, name):
    bar = alt.Chart(df).mark_bar().encode(
            y = alt.X(feature,
                axis = alt.Axis(
                title = feature)),
            x = alt.Y('county',
                axis = alt.Axis(
                    title = 'Counties',
                    labels = False,
                    ticks= False),
                sort = alt.EncodingSortField(field=feature, order = 'descending')),
            color = alt.Color('county',
                sort= [feature],
                legend=alt.Legend(
                    title = ''))
        ).properties(
            height=300,
            width=450,
            title = {
                'text' :[feature + ' of ' + name],
            },
        ).configure_title(
            fontSize=14,
            anchor='start',
            subtitlePadding = 2,
            dy=-10
        )
    return bar

# features = bottom3_dem.columns.values[1:]
input_df = [bottom3_dem, nominated_dem]
features = ['Population Density', 'Median Age', 'Vacant Housing Units','Median Household Income', 'Population Growth']
name = ['Lower-ranked Counties', 'Nominated Counties']


In [15]:
# you can change each input to see each demographic factor in different county categories

# here as an example
# plot the Population Growth feature in the bottom3_dem df
# and name the title of chart as Lower-ranked Counties
plot_dem_features(input_df[0], features[-1], name[0])

# plot the Population Growth feature in the nominated_dem df
# plot_dem_features(input_df[1], features[-1], name[0])

alt.Chart(...)

#### Findings about population growth rate:

Diving into each demographic factor at county level, we found the main contribution to the average population growth rate in the bottom3 (lower-ranked) counties came from the counties in Texas: Hays county (10.1%), Collin county (9.1%), and Travis county (6.6%). The contributions from the counties in California and New York states are close to zero (0.3%) in two counties, and all others are negative.

### Make an aggregated demographic dataframe for comparing the nominated, top3, and bottom3 counties

In [16]:
# get the ggregated mean value
def agg_demo_diff(df):
    mean_df = np.round(df.mean(), decimals = 3)
    demo_df = pd.DataFrame(data = mean_df, columns = ['values'])
    demo_df.reset_index(inplace=True)
    demo_df.rename(columns = {'index': 'county'}, inplace=True)
    
    return demo_df

agg_bottom3_demo = agg_demo_diff(bottom3_dem)
agg_nominated_demo = agg_demo_diff(nominated_dem)

# merge nominated and bottom3 together
demo_df = agg_nominated_demo.merge(agg_bottom3_demo, on='county')
demo_df.columns = ['demographic variables', 'nominated','bottom3']

# calculate the different of nominated and bottom3 
# in ratio to overcome the different value scale
# to calcuate how many time the nominated counties are higher/lower than  bottom3 counties
demo_df['difference'] = round((demo_df['nominated'] / demo_df['bottom3']-1), 3) 

In [17]:
demo_df

,demographic variables,nominated,bottom3,difference
0,Population Density,7301.440,9390.421,-0.222
1,Median Age,34.640,37.978,-0.088
2,Vacant Housing Units,0.064,0.060,0.067
3,Median Household Income,58302.600,102362.889,-0.430
4,Population Growth,0.053,0.027,0.963


Make a bar chart using Altair to see the differences between the nominated and the lower-ranked (bottom3)

In [18]:
def plot_dem_diff():
    bars = alt.Chart(demo_df).mark_bar().encode(
        x=alt.X('demographic variables:N',
            axis = alt.Axis(
                title = 'Demographic variables',
                labels = False,
                ticks = False),
            sort = alt.EncodingSortField(field='difference', order = 'descending'),
        ),
        y = alt.Y('difference:Q',
            axis= alt.Axis(
                title = 'Different (%)',
                ticks=False),
        ),
        color = alt.Color('demographic variables:N',
            sort = ['difference'],
            legend=alt.Legend(
                title = ''),
        )
    ).properties(
        height=300,
        width=450,
        title = {
            'text' :["Demographic Difference Between",
            "Nominated Counties and Lower-ranked Counties"]},
    ).configure_title(
        fontSize=14,
        anchor='start',
        subtitlePadding = 2,
        dy=-10
    )

    return bars

plot_dem_diff()

alt.Chart(...)

#### Findings: 

In order to see if there are some demographic differences between the nominated counties and the lower-ranked counties, we made a bar chart to show the percentage differences between them.

Compared with the lower-ranked counties we found that, on average, nominated counties have higher population growth rate and vacant housing units. They also had lower median age, population density, and median household income. Among these factors, the average population growth rate of the nominated counties is almost twice that of the lower-ranked counties.

In general, based on the differences, **areas with higher population growth rate but lower population density, younger communities and relatively lower median household income, seem to be the attributes for more profitable counties for rental property investors.** 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=718fdf0e-933b-4ec5-90a8-5b2fe887b720' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>